In [1]:
import os

In [2]:
%pwd

'/Users/karthikaselvaraj/Desktop/test/Sports_Classification/research'

In [3]:
# os.chdir("/Users/karthikaselvaraj/Desktop/test/Sports_Classification")
os.chdir('../')

In [4]:
%pwd

'/Users/karthikaselvaraj/Desktop/test/Sports_Classification'

In [5]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [6]:
from sports_classifier.constants import *
from sports_classifier.utils.common import read_yaml, create_directories

In [7]:
# Configuration Manager
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
# components
import os
import urllib.request as request
import zipfile
from sports_classifier import logger
from sports_classifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

            

In [9]:
# pipeline 
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

True
True
[2023-11-09 22:01:18,757: INFO: common: yaml file config/config.yaml loaded successfully]
[2023-11-09 22:01:18,759: INFO: common: yaml file params.yaml loaded successfully]
[2023-11-09 22:01:18,760: INFO: common: Created directory at : artifacts]
[2023-11-09 22:01:18,760: INFO: common: Created directory at : artifacts/data_ingestion]
[2023-11-09 22:01:24,832: INFO: 1052934199: artifacts/data_ingestion/data.zip download! with following info Connection: close
Content-Length: 24862744
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f2bbc0f962c7b661517baac865b4a2395eee83970d48cf8894c48594ec7f69f1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3AA6:2FEFB1:278D5:3AC88:654D0956
Accept-Ranges: bytes
Date: Thu, 09 Nov 2023 16:31:21 GMT
Via: 1.1 varnish
X-Served-By: cache-del21736-DEL
X-C